In [169]:
import datetime
import numpy as np
import pandas as pd
import yfinance as yf
import warnings
from scipy.stats import pearsonr
warnings.filterwarnings('ignore')

In [170]:
def load_data(ticker_symbol ,s_date, e_date):
    stock_data = yf.download(ticker_symbol, start=s_date, end=e_date)
    stock_data = stock_data[['Adj Close']].reset_index()
    date_all = pd.date_range(s_date,e_date,freq='D').to_frame().rename(columns = {0:'Date'}).reset_index(drop=True)
    price_df = pd.merge(date_all,stock_data,how='left')
    price_df = price_df.fillna(method='ffill').dropna()
    return price_df

In [171]:
def price_ret(price_df,method):
    if method == 'Y':
        # print("연간 수익률")
        price_year = price_df.reset_index(drop=True).iloc[0::365,:]
        year_ret = price_year.set_index(['Date']).pct_change().dropna()
        return year_ret
    elif method == 'M':
        # print("월간 수익률")
        price_month = price_df.reset_index(drop=True).iloc[0::30,:]
        month_ret = price_month.set_index(['Date']).pct_change().dropna()
        return month_ret
    elif method == 'W':
        # print("주간 수익률")
        price_df = price_df.reset_index(drop= True)
        price_df['Day_Name'] = price_df['Date'].dt.day_name()
        price_mon = price_df[price_df['Day_Name'] == price_df['Day_Name'][0]].set_index(['Date']).drop(['Day_Name'], axis=1)
        week_ret = price_mon.pct_change(1).dropna()
        return week_ret
    elif method == 'D':
        # print("일별 수익률")
        price_day = price_df.set_index(['Date'])
        day_ret = price_day.pct_change(1).dropna()
        return day_ret
    else : 
        print(f"연속{method}일 수익률")
        price_df = price_df.set_index(['Date'])
        return price_df.pct_change(method).dropna()

In [172]:
def pc_corr(method = 'P'):
    c_inf = ['395400.ks' , '330590.ks','357120.ks','404990.ks','417310.ks']
    p_inf = ['034730.ks' , '023530.ks','267250.ks','006730.kq','086790.ks']
    ko_inf = ['^KS11','^KQ11']
    corr = pd.DataFrame()
    p_val = pd.DataFrame()
    
    # 전체기간
    s_date = [datetime.datetime(2022,4,1),datetime.datetime(2022,7,1),datetime.datetime(2023,2,1)]
    e_date = [datetime.datetime(2024,4,1),datetime.datetime(2022,12,31),datetime.datetime(2023,7,31)]
    col_name = ['전체','금리 상승기','동결기']
    for d in range(len(s_date)):
        c_temp = []
        p_temp = []   
        for i in range(len(c_inf)):
            c_symbol = c_inf[i]
            if method == 'KS':
                p_symbol = ko_inf[0]
            elif method == 'KQ' :
                p_symbol = ko_inf[1]
            else :
                p_symbol = p_inf[i]
            c_df = load_data(c_symbol,s_date[d],e_date[d])
            p_df = load_data(p_symbol,s_date[d],e_date[d])     
            c_ret = price_ret(c_df,'D')
            p_ret = price_ret(p_df,'D')
            result = pearsonr(list(c_ret['Adj Close']),list(p_ret['Adj Close']))
            c_temp.append(result[0])
            p_temp.append(result[1])
        corr[col_name[d]] = c_temp
        p_val[col_name[d]] = p_temp

    
    corr.index = ['SK리츠','롯데리츠','코람코라이프인프라리츠','신한서부티앤디리츠','코람코더원리츠']
    p_val.index = ['SK리츠','롯데리츠','코람코라이프인프라리츠','신한서부티앤디리츠','코람코더원리츠']

    return corr,p_val

In [173]:
s_corr,s_p_val=pc_corr()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [174]:
ks_corr,ks_p_val=pc_corr('KS')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [175]:
kq_corr,kq_p_val=pc_corr('KQ')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [176]:
corr_df = pd.concat([ks_corr,kq_corr,s_corr],axis = 1)
p_val_df = pd.concat([ks_p_val,kq_p_val,s_p_val],axis = 1)

In [177]:
column_name = ['kospi_전체','kospi_금리상승기','kospi_금리동결기','kosdaq_전체','kosdaq_금리상승기','kosdaq_금리동결기','스폰서_전체','스폰서_금리상승기','스폰서_금리동결기']
corr_df.columns = column_name
p_val_df.columns = column_name

In [160]:
corr_df.to_csv('대주주_상관계수.csv')

In [161]:
p_val_df.to_csv('대주주_P-value.csv')